In [ ]:
# Step1: Determind DA-Price

import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
import pandas as pd 
from data import L_D_Hours, Wind_Hours, PC_DA, P_max, Load_Demand, Cost_Generation, Wind_Farm_Power, Wind, T, N, P_min # Import data from data.py

# Load bid data from a CSV file
file_path = "day_ahead_bids.csv"
df = pd.read_csv(file_path)

# Defining the number of demand nodes and wind farms 
Dem = len(L_D_Hours)  # Number of demand nodes
Wind = len(Wind_Hours)  # Number of wind farms
Gen = len(P_max)  # Number of generators

# Ensure correct column names
df.columns = ["Hour", "Bid Price", "Bid Amount"]

# Convert bid data into a dictionary grouped by hour
bid_data = {}
for hour, group in df.groupby("Hour"):
    bid_data[hour] = group[["Bid Price", "Bid Amount"]].to_dict(orient="records")

# Create a bid price map for each demand node (1 to Dem) for each hour
bid_prices = {} 
for hour in range(1, T + 1):
    # Get the sorted bids by price for the current hour
    sorted_bids = sorted(bid_data.get(hour, []), key=lambda x: x["Bid Price"])
    
    # Allocate bid prices to demand nodes based on sorted bids
    total_bid_amount = 0
    bid_prices[hour] = []
    
    for bid in sorted_bids:
        bid_price = bid["Bid Price"]
        bid_amount = bid["Bid Amount"]
        
        # Allocate the bid price to demand nodes 
        for _ in range(int(bid_amount)):  
            bid_prices[hour].append(bid_price)
    
    if len(bid_prices[hour]) < Dem:
        # Fill the remaining demand nodes with the highest available price
        highest_price = sorted_bids[-1]["Bid Price"]
        bid_prices[hour].extend([highest_price] * (Dem - len(bid_prices[hour])))

h = 7  

# Model
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)

# Load and wind profile for hour `h`
Load = np.zeros(Dem)
Load[:] = L_D_Hours[:, h]     # load demand at hour `h`
Wind_profile = np.zeros(Wind)
Wind_profile[:] = Wind_Hours[:, h]  # Wind production at hour `h`

# Decision Variables
model.P_g = Var(range(Gen), domain=NonNegativeReals)  # Production of conventional generators
model.P_w = Var(range(Wind), domain=NonNegativeReals)  # Production of wind farms
model.P_d = Var(range(Dem), domain=NonNegativeReals)  # Demand

# Objective function: social welfare
def objective_rule(model):
    return sum(bid_prices[h][dem] * model.P_d[dem] for dem in range(Dem)) - \
           sum(PC_DA[gen] * model.P_g[gen] for gen in range(Gen)) - \
           sum(0 * model.P_w[wind] for wind in range(Wind))  

model.obj = Objective(rule=objective_rule, sense=maximize)

# Constraints

# Capacity constraints for generators
def conventional_generation_rule(model, gen):
    return model.P_g[gen] <= P_max[gen]

model.conventional_generation = Constraint(range(Gen), rule=conventional_generation_rule)

# Capacity constraints for wind 
def wind_generation_rule(model, wind):
    return model.P_w[wind] <= Wind_profile[wind]  # Wind generation cannot exceed the profile

model.wind_generation = Constraint(range(Wind), rule=wind_generation_rule)

# Demand constraint
def demand_rule(model, dem):
    return model.P_d[dem] <= L_D_Hours[dem, h]  # Use the correct hour's demand

model.demand = Constraint(range(Dem), rule=demand_rule)

# Balance constraint (Total demand = total generation)
def balance_rule(model):
    return sum(model.P_d[dem] for dem in range(Dem)) == \
           sum(model.P_g[gen] for gen in range(Gen)) + \
           sum(model.P_w[wind] for wind in range(Wind))

model.balance = Constraint(rule=balance_rule)

# Solve the model using GLPK and ensuring dual values are available
solver = SolverFactory('glpk')
solver.solve(model)

# Solution
if model.obj() is not None:

    # Production of wind farms
    for wind in range(Wind):
        print(f"Production of Wind farm (before balancing) # {wind}: {model.P_w[wind].value} (MW)")
    print("")

    # Production of generators
    for gen in range(Gen):
        print(f"Production of Conventional generator (before balancing) # {gen}: {model.P_g[gen].value} (MW)")
    print("")

    # Market Clearing Price
    MC_price = model.dual[model.balance]  


    # Profit for generators
    Revenue_p_g = [int(MC_price * model.P_g[gen].value) for gen in range(Gen)]
    Cost_p_g = [int(PC_DA[gen] * model.P_g[gen].value) for gen in range(Gen)]
    Profit_p_g = [Revenue_p_g[gen] - Cost_p_g[gen] for gen in range(Gen)]


    # Profit for wind farms
    Revenue_p_wf = [int(MC_price * model.P_w[wind].value) for wind in range(Wind)]
    Cost_p_wf = [int(0 * model.P_w[wind].value) for wind in range(Wind)]  
    Profit_p_wf = [Revenue_p_wf[wind] - Cost_p_wf[wind] for wind in range(Wind)]
    
   
    # Profit for demand
    Revenue_Demand = [int(model.P_d[dem].value * bid_prices[h][dem]) for dem in range(Dem)]
    Cost_Demand = [int(model.P_d[dem].value * MC_price) for dem in range(Dem)]
    Profit_Demand = [Revenue_Demand[dem] - Cost_Demand[dem] for dem in range(Dem)]


# Step 2: Balancing Market

# Day-ahead forecasts
DA_price = MC_price
pg_DA = np.array([model.P_g[gen].value for gen in range(Gen)])  # Generator power at hour h
pw_DA = np.array([model.P_w[wind].value for wind in range(Wind)])  # Wind farm power at hour h
DA_P = sum(pg_DA) + sum(pw_DA)  # Total day-ahead production

# Shutting down generator
OFF = 9  # Manually selecting generator 10 to shut down 
P_g = pg_DA.copy()
P_g[OFF] = 0  # Generator OFF has a production of 0
P_max[OFF] = 0  # Generator OFF has a max capacity of 0

loss = [sum(pg_DA) - sum(P_g)]
gain = []

# Balancing cost parameters
Per_Up = 0.10
Per_Down = 0.15
C_curt = 500
C_up = {gen: DA_price + Per_Up * PC_DA[gen] for gen in range(Gen)}
C_down = {gen: DA_price - Per_Down * PC_DA[gen] for gen in range(Gen)}

# Adjust wind farm productions based on efficiency changes
P_w = pw_DA.copy()
for wind in range(Wind):
    if wind < 3:
        eff = 0.15
        loss.append(P_w[wind] * eff)
        P_w[wind] *= (1 - eff)
    else:
        eff = 0.1
        gain.append(P_w[wind] * eff)
        P_w[wind] *= (1 + eff)

# Actual production before balancing market
P_tot = sum(P_g) + sum(P_w)
Delta_P = DA_P - P_tot

# Define model
model_bal = ConcreteModel()
model_bal.dual = Suffix(direction=Suffix.IMPORT)

# Decision Variables
model_bal.P_g = Var(range(Gen), domain=NonNegativeReals)  # Production of conventional generators
model_bal.Gen_up = Var(range(Gen), domain=NonNegativeReals)
model_bal.Gen_down = Var(range(Gen), domain=NonNegativeReals)
model_bal.Load_curt = Var(range(Dem), domain=NonNegativeReals)

# Objective function: minimize balancing cost
def objective_rule(model_bal):
    return (
        sum(C_up[gen] * model_bal.Gen_up[gen] for gen in range(Gen)) -
        sum(C_down[gen] * model_bal.Gen_down[gen] for gen in range(Gen)) +
        C_curt * sum(model_bal.Load_curt[dem] for dem in range(Dem))
    )

model_bal.obj = Objective(rule=objective_rule, sense=minimize)

# Balancing constraint
def balance_rule(model_bal):
    return (
        Delta_P ==
        sum(model_bal.Gen_up[gen] for gen in range(Gen)) -
        sum(model_bal.Gen_down[gen] for gen in range(Gen)) +
        sum(model_bal.Load_curt[dem] for dem in range(Dem))
    )

model_bal.balance = Constraint(rule=balance_rule)

# Generator regulation limits
model_bal.generation_up_limits = ConstraintList()
model_bal.generation_down_limits = ConstraintList()
for gen in range(Gen):
    model_bal.generation_up_limits.add(model_bal.Gen_up[gen] <= P_max[gen] - P_g[gen])
    model_bal.generation_down_limits.add(model_bal.Gen_down[gen] <= P_g[gen])

# Demand curtailment limits
model_bal.demand_curtailment_limits = ConstraintList()
for dem in range(Dem):
    model_bal.demand_curtailment_limits.add(model_bal.Load_curt[dem] <= L_D_Hours[dem, h])

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model_bal)

# Print Results
if model_bal.obj() is not None:
    print(f"Optimal Balancing Cost: {model_bal.obj()} $")
    for gen in range(Gen):
        if (model_bal.Gen_up[gen]() is not None and model_bal.Gen_up[gen]() > 0.01) or (model_bal.Gen_down[gen]() is not None and model_bal.Gen_down[gen]() > 0.01):
            print(f"Generator {gen} - Up: {model_bal.Gen_up[gen].value:.2f} MW, Down: {model_bal.Gen_down[gen].value:.2f} MW")
    print(f"Load curtailment: {sum(model_bal.Load_curt[dem].value for dem in range(Dem)):.2f} MW")
    print(f"Balancing price: {model_bal.dual[model_bal.balance]:.2f} $/MWh")
else:
    print("No optimal solution available")

# Solution
if model_bal.obj() is not None:

    # Setting up price schemes
    balance_price = model_bal.dual[model_bal.balance]
    one_price = balance_price
    two_price_gain = DA_price
    two_price_loss = balance_price

   # Printing the total loss in production and profits for Generator OFF
print(f"\n--- Generator {OFF} ---")
print(f"Loss in production: {round(loss[0])} MW")
print(f"Day-Ahead Profit: {round((DA_price - PC_DA[OFF]) * pg_DA[OFF])} $")
print(f"One-Price Scheme Profit: {round(-loss[0] * one_price)} $")
print(f"Two-Price Scheme Profit: {round(-loss[0] * two_price_loss)} $")


# Loop through each generator
for gen in range(Gen):
    if gen == 9:  # Skip generator 10
        continue
    # Check if the generator has significant up or down values
    if (model_bal.Gen_up[gen].value is not None and model_bal.Gen_up[gen].value > 0.01) or \
       (model_bal.Gen_down[gen].value is not None and model_bal.Gen_down[gen].value > 0.01):
        
        # Calculate the profit (day-ahead, one-price, two-price)
        day_ahead_profit = round((DA_price - PC_DA[gen]) * pg_DA[gen])
        one_price_profit = round((one_price) * (model_bal.Gen_up[gen].value - model_bal.Gen_down[gen].value))

        # In the two-price scheme, the deficit is caused by imbalances
        two_price_deficit = round(balance_price * (model_bal.Gen_down[gen].value))  # Deficit for causing imbalances
        two_price_profit = round(balance_price * (model_bal.Gen_up[gen].value - model_bal.Gen_down[gen].value))  # Profit from balancing services
        
        print(f"\n--- Generator {gen} ---")
        print(f"Generator {gen} - Up: {round(model_bal.Gen_up[gen].value)} MW, Down: {round(model_bal.Gen_down[gen].value)} MW")
        print(f"Day-Ahead Profit: {day_ahead_profit} $")
        print(f"One-Price Scheme Profit: {one_price_profit} $")
        print(f"Two-Price Scheme Deficit (Imbalance): {two_price_deficit} $")
        print(f"Two-Price Scheme Profit (Balancing): {two_price_profit} $")

    # If the generator is producing power, print day-ahead details
    elif pg_DA[gen] > 0:
        day_ahead_profit = round(DA_price * pg_DA[gen])
        one_price_profit = round((one_price) * (model_bal.Gen_up[gen].value - model_bal.Gen_down[gen].value))

        # For two-price scheme
        two_price_deficit = round(balance_price * (model_bal.Gen_down[gen].value))  # Deficit for causing imbalances
        two_price_profit = round(DA_price * (model_bal.Gen_up[gen].value - model_bal.Gen_down[gen].value))  # Profit from balancing services
        
        print(f"\n--- Generator {gen} ---")
        print(f"Up: {round(model_bal.Gen_up[gen].value)} MW, Down: {round(model_bal.Gen_down[gen].value)} MW")
        print(f"Day-Ahead Profit: {day_ahead_profit} $")
        print(f"One-Price Scheme Profit: {one_price_profit} $")
        print(f"Two-Price Scheme Deficit (Imbalance): {two_price_deficit} $")
        print(f"Two-Price Scheme Profit (Balancing): {two_price_profit} $")


# Printing for each wind turbine
for wind in range(Wind):
    print(f"\n--- Wind Turbine {wind} ---")
    
    # Day-ahead profit calculation for wind turbines
    day_ahead_profit_wind = round(DA_price * pw_DA[wind])
    
    # One-price and two-price scheme calculations based on wind turbine
    if wind < 3:
        one_price_profit_wind = round(-loss[wind + 1] * one_price)
        two_price_profit_wind = round(-loss[wind + 1] * two_price_loss)
        total_one_price_profit = round(day_ahead_profit_wind - loss[wind + 1] * one_price)
        total_two_price_profit = round(day_ahead_profit_wind - loss[wind + 1] * two_price_loss)
    else:
        one_price_profit_wind = round(gain[wind - 3] * one_price)
        two_price_profit_wind = round(gain[wind - 3] * two_price_gain)
        total_one_price_profit = round(day_ahead_profit_wind + gain[wind - 3] * one_price)
        total_two_price_profit = round(day_ahead_profit_wind + gain[wind - 3] * two_price_gain)
    
    print(f"Day-Ahead Profit: {day_ahead_profit_wind} $")
    print(f"One-Price Scheme Profit: {total_one_price_profit} $")
    print(f"Two-Price Scheme Profit: {total_two_price_profit} $")

print("\n--- Generator Production After Balancing ---")
for gen in range(Gen):
    balanced_production = P_g[gen] + model_bal.Gen_up[gen].value - model_bal.Gen_down[gen].value
    print(f"Generator {gen} Production (After Balancing): {balanced_production:.2f} MW")

print("\n--- Wind Farm Production After Balancing ---")
for wind in range(Wind):
    print(f"Wind Farm {wind} Production (After Balancing): {P_w[wind]:.2f} MW")

# Handling if no optimal solution is available
optimal_solution_available = model_bal.obj() is not None
if not optimal_solution_available:
    print("No optimal solution available")

Production of Wind farm (before balancing) # 0: 143.1758086 (MW)
Production of Wind farm (before balancing) # 1: 146.418634 (MW)
Production of Wind farm (before balancing) # 2: 87.212025 (MW)
Production of Wind farm (before balancing) # 3: 121.150513 (MW)
Production of Wind farm (before balancing) # 4: 127.118818 (MW)
Production of Wind farm (before balancing) # 5: 103.6143558 (MW)

Production of Conventional generator (before balancing) # 0: 0.0 (MW)
Production of Conventional generator (before balancing) # 1: 0.0 (MW)
Production of Conventional generator (before balancing) # 2: 0.0 (MW)
Production of Conventional generator (before balancing) # 3: 0.0 (MW)
Production of Conventional generator (before balancing) # 4: 0.0 (MW)
Production of Conventional generator (before balancing) # 5: 155.0 (MW)
Production of Conventional generator (before balancing) # 6: 155.0 (MW)
Production of Conventional generator (before balancing) # 7: 400.0 (MW)
Production of Conventional generator (before bal